In [216]:
## Import all the things. 
import numpy as np
import csv
import pandas as pd
import random
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import opinion_lexicon as op
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk import pos_tag
np.set_printoptions(linewidth=400)   # optional: widens column of numpy array display

In [217]:
## get positve and negative words from the lexicon. 
pos_words = set(op.words('positive-words.txt'))
neg_words = set(op.words('negative-words.txt'))

In [218]:
## create lists for response and target. 
X = [] ; y = []; initial_set = []; X_test = []; y_test = []
with open("reviews.txt", 'r', encoding='utf8') as reviews:
    for review in reviews:
        if review != "":
            review = review.strip()
            review = review.lower()
            #review = word_tokenize(review)
            if len(review) > 1:
                initial_set.append(review)
    random.shuffle(initial_set)
num_sentences = len(initial_set)

In [219]:
# write out a split to files:
train_len = int(0.8 * num_sentences)
test_len = num_sentences - train_len

train_list = initial_set[:train_len]
test_list = initial_set[train_len:]
print(len(train_list))
print(len(test_list))

with open('reviews-train.txt', 'w') as r_train:
    for line in train_list:
        line = str(line)
        r_train.write(line)
        r_train.write('\n')
with open('reviews-test.txt', 'w') as r_test:
    for test_line in test_list:
        test_line = str(test_line)
        r_test.write(test_line)
        r_test.write('\n')

1999
500


In [220]:
def return_rating(pos, neg):
        stars = [0,0,0,0,0]
        if negCount > (posCount*2):
            stars[0] = 1
        elif posCount > (negCount*2):
            stars[4] = 1
        elif negCount == posCount:
            stars[2] = 1
        elif negCount < posCount:
            stars[3]= 1
        elif posCount < negCount:
            stars[1] = 1
            

In [221]:
rev_train_file = open('reviews-train.txt')
lines = rev_train_file.readlines()
for line in lines:
    line = line.strip()
    X.append(word_tokenize(line[:-1]))
    y.append(line[-1])
rev_train_file.close()

In [222]:
rev_test_file = open('reviews-test.txt')
lines = rev_test_file.readlines()
for line in lines:
    line = line.strip()
    X_test.append(word_tokenize(line[:-1]))
    y_test.append(line[-1])
rev_test_file.close()

In [223]:
def pos_neg_count(line):
    posCount = 0; negCount = 0
    for word in line:
        if word in pos_words:
            posCount +=1
        elif word in neg_words:
            negCount+=1
    return posCount, negCount
    

In [227]:
def superlative_count(tokenized):
    try:
        for i in tokenized[:3]:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            #print(tagged)

    except Exception as e:
        pass
    count = 0
    for x in tagged:
        if (x[1]) == 'JJS':
            count +=6
        if (x[1]) == 'JJR':
            count +=3
        if (x[1]) == 'JJ':
            count +=1    
        
    return count

In [228]:
###################################################################################################################
############################################ FEATURE EXTRACTION ###################################################
###################################################################################################################
def np_feature_extraction(X,y):
    X_list = [] ; y_list = []

    punktLine = PunktSentenceTokenizer()
    for x,line in enumerate(X):
        ### GET TARGET #####
        rating = int(y[x])
        ### GET LENGTH ####
        leng = len(line)
        ### GET POS AND NEG COUNTS
        posCount, negCount = pos_neg_count(line)
        
        sent = ' '.join(line)
        token = punktLine.tokenize(sent)
        JJS = superlative_count(token)
        print(JJS)
        
        
        ## THIS IS THE NUMPY LINE WITH FEATURES
        line = [leng, posCount, negCount, JJS]
        
        ### APPEND LINES #####
        X_list.append(line)
        y_list.append(rating)
        
    ##TURN INTO NUMPY ARRAYS
    X_np = (np.array(X_list))
    y_np = (np.array(y_list))
    
    return X_np,y_np
#########################################################################################################################

In [229]:
X_train_np, y_train_np = np_feature_extraction(X,y)
print(X_train_np)
X_test_np, y_test_np = np_feature_extraction(X_test, y_test)


2
0
0
5
0
1
7
5
5
2
5
4
1
1
1
0
5
1
1
1
4
2
0
0
0
3
1
1
0
1
0
1
3
1
2
2
0
3
0
1
3
0
0
0
0
2
1
1
2
0
9
3
0
0
1
6
3
2
13
7
1
1
3
2
2
3
0
3
5
5
9
3
2
0
0
1
0
1
7
0
3
0
0
0
3
1
0
4
3
5
0
1
1
3
1
2
7
1
4
2
0
0
2
4
1
1
3
1
4
3
1
0
2
2
0
0
1
25
6
1
0
2
7
0
0
0
1
1
1
5
0
3
7
4
5
0
10
1
2
3
2
1
3
3
1
1
1
1
4
2
1
0
1
1
1
1
1
4
1
1
12
2
3
2
0
6
2
8
1
4
0
1
1
0
0
2
4
1
0
1
2
0
0
1
2
1
4
5
1
1
1
1
0
3
0
2
1
2
1
0
2
0
1
0
0
1
2
1
2
2
0
8
1
2
0
0
5
4
3
2
0
0
1
0
4
0
4
2
1
0
2
1
1
0
1
4
1
7
0
0
0
1
0
5
0
4
1
1
5
2
0
1
0
4
6
12
0
4
7
1
0
1
4
3
0
1
0
4
1
1
2
3
5
9
3
0
0
1
2
0
2
1
1
2
3
2
2
2
1
1
3
1
4
0
1
1
2
7
1
3
4
2
0
5
0
2
1
3
4
0
0
0
1
3
6
4
1
8
2
5
4
2
7
3
0
6
1
0
4
4
3
7
2
3
0
1
14
1
2
2
0
2
0
1
1
6
1
6
6
1
6
3
1
13
4
1
0
2
3
3
2
0
0
1
3
0
3
4
1
1
4
2
1
1
13
0
4
0
2
2
5
2
9
0
8
3
0
1
3
4
1
1
3
2
1
0
4
1
2
0
5
1
3
1
4
2
0
1
3
2
1
4
3
1
31
2
0
0
0
1
5
5
3
1
4
1
5
0
5
1
3
2
2
1
5
1
2
4
0
0
4
2
3
1
1
3
0
1
3
4
36
3
2
1
12
1
0
2
2
1
4
26
0
1
1
1
1
2
3
4
0
4
11
2
2
1
1
1
4
10
0
1
4
0
6
5
0
1
7
3
1
2
2


In [230]:
for mclass in ('multinomial', 'ovr'):
    lr = LogisticRegression(solver='lbfgs', max_iter=3000, random_state=0, multi_class=mclass).fit(X_train_np, y_train_np)
    yhat = lr.predict(X_test_np)
     
    # the 3 lines below show how to invoke various output    
    print("\n",mclass,"Accuracy",accuracy_score(y_test_np, yhat))
    print("\n",mclass,"Classification Report\n",classification_report(y_test_np, yhat),sep="")
    print("\n",mclass,"Classification Report\n",confusion_matrix(y_test_np, yhat),sep="")


 multinomial Accuracy 0.676

multinomialClassification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        17
           3       0.00      0.00      0.00        35
           4       0.00      0.00      0.00        96
           5       0.69      0.99      0.81       342

   micro avg       0.68      0.68      0.68       500
   macro avg       0.14      0.20      0.16       500
weighted avg       0.47      0.68      0.55       500


multinomialClassification Report
[[  0   0   0   0  10]
 [  0   0   0   1  16]
 [  0   0   0   0  35]
 [  1   0   1   0  94]
 [  0   0   0   4 338]]

 ovr Accuracy 0.674

ovrClassification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        17
           3       0.00      0.00      0.00        35
           4       0.00      0.00 

/home/loki/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/loki/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
